## Required Functions

In [6]:
import numpy as np
import pandas as pd
import pickle
import re
import matplotlib.pyplot as plt

In [7]:
def unpickle(file):
    with open(file, 'rb') as fo:
        direc = pickle.load(fo, encoding='bytes')
    return direc

## Load CIFAR-10 Data

In [8]:
pd_tr = pd.DataFrame()
tr_y = pd.DataFrame()
 
for i in range(1,6):
    data = unpickle('data/cifar-10-batches-py/data_batch_' + str(i))
    pd_tr = pd_tr.append(pd.DataFrame(data[b'data']))
    tr_y = tr_y.append(pd.DataFrame(data[b'labels']))
    pd_tr['labels'] = tr_y 

tr_x = np.asarray(pd_tr.iloc[:, :3072])
tr_y = np.asarray(pd_tr['labels'])
ts_x = np.asarray(unpickle('data/cifar-10-batches-py/test_batch')[b'data'])
ts_y = np.asarray(unpickle('data/cifar-10-batches-py/test_batch')[b'labels'])    
labels = unpickle('data/cifar-10-batches-py/batches.meta')[b'label_names']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [9]:
pd_tr.head()

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,labels
0,59,43,50,68,98,119,139,145,149,149,...,58,65,59,46,57,104,140,84,72,6
1,154,126,105,102,125,155,172,180,142,111,...,42,67,101,122,133,136,139,142,144,9
2,255,253,253,253,253,253,253,253,253,253,...,83,80,69,66,72,79,83,83,84,9
3,28,37,38,42,44,40,40,24,32,43,...,39,59,42,44,48,38,28,37,46,4
4,170,168,177,183,181,177,181,184,189,189,...,88,85,82,83,79,78,82,78,80,1


[Python中iloc如何使用](https://blog.csdn.net/qq1483661204/article/details/77587881)

## Display the images

In [10]:
def plot_CIFAR(ind):
    arr = tr_x[ind]
    R = arr[0:1024].reshape(32,32)/255.0
    G = arr[1024:2048].reshape(32,32)/255.0
    B = arr[2048:].reshape(32,32)/255.0
 
    img = np.dstack((R,G,B))
#     title = re.sub('[!@#$b]', '', str(labels[tr_y[ind]]))
    name = str(labels[tr_y[ind]])
    title = name[1:]
    fig = plt.figure(figsize=(3,3))
    ax = fig.add_subplot(111)
    ax.imshow(img, interpolation='bicubic')
    ax.set_title('Category = '+ title, fontsize=15)

In [11]:
def View_data(number):
    plot_CIFAR(number)

 ## Cifar 10 Classes:
- 0: airplane
- 1: automobile(car)
- 2: bird
- 3: cat
- 4: deer
- 5: dog
- 6: frog
- 7: horse
- 8: ship
- 9: truck

In [12]:
from ipywidgets import interact
interact(View_data, number=(0,49999));

interactive(children=(IntSlider(value=24999, description='number', max=49999), Output()), _dom_classes=('widge…

## Before building CNN

In [13]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


### Data preprocessing

In [14]:
tr_x.shape

(50000, 3072)

In [15]:
x_train0 = tr_x.reshape(50000,32,32,3)
x_test0 = ts_x.reshape(10000,32,32,3) 

In [16]:
x_train = x_train0/255
x_test = x_test0/255

In [17]:
x_train[34].shape

(32, 32, 3)

1-hot encoding 

In [18]:
from keras.utils import np_utils

In [19]:
y_train = np_utils.to_categorical(tr_y,10)
y_test = np_utils.to_categorical(ts_y,10)

## Building CNN

In [21]:
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, concatenate
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import SGD, Adadelta

The structure of our artificial CNN networks.

![](image/CNN_structure.PNG)

- $W_{4\times4}$ denotes a 4 by 4 filter matrix
- $P_{2\times2}$ denotes a 2 by 2 Max pooling matrix

### Define the hidden layers

In [103]:
f1 = Conv2D(8,(5,5), padding = 'same', activation = 'selu') # Number of filters: 8
pool1 = MaxPool2D(pool_size = (2, 2))

f2 = Conv2D(16,(5,5), padding = 'same', activation = 'selu') # Number of filters: 16
pool2 = MaxPool2D(pool_size = (2, 2))

f3 = Conv2D(32,(5,5), padding = 'same', activation = 'selu') # Number of filters: 32
pool3 = MaxPool2D(pool_size = (2, 2))

f4 = Dense(200, activation='relu')  
f5 = Dense(100, activation='selu')     #   50 to 20
f6 = Dense(100, activation='selu')     #   50 to 20
f7 = Dense(10, activation='softmax')   #   40 to 10

### Define the variables

In [104]:
x = Input(shape = (32, 32, 3))
h1 = pool1(f1(x))
h2 = pool2(f2(h1))
h3 = pool3(f3(h2))
h4 = Flatten() (f4(h3))
h5 = f5(h4)
h6 = f6(h4)

In [105]:
u = concatenate([h5,h6]) 
y = f7(u)

In [106]:
model = Model(x,y)

In [107]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 32, 32, 8)    608         input_15[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_46 (MaxPooling2D) (None, 16, 16, 8)    0           conv2d_46[0][0]                  
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 16, 16, 16)   3216        max_pooling2d_46[0][0]           
__________________________________________________________________________________________________
max_poolin

In [108]:
model.compile(loss = 'mse', optimizer = SGD(lr = 0.7), metrics=['accuracy']) # 0.625
# model.compile(loss = 'mse', optimizer = Adadelta(lr = 1), metrics=['accuracy'])

### Training 

In [113]:
model_fitted = model.fit(x_train, y_train, batch_size = 100, epochs = 5)

Epoch 1/5
50000/50000 [==============================] - 18s 364us/step - loss: 0.0314 - acc: 0.7891
Epoch 2/5
50000/50000 [==============================] - 18s 365us/step - loss: 0.0310 - acc: 0.7920
Epoch 3/5
50000/50000 [==============================] - 19s 383us/step - loss: 0.0308 - acc: 0.7939
Epoch 4/5
50000/50000 [==============================] - 19s 372us/step - loss: 0.0297 - acc: 0.8022
Epoch 5/5
50000/50000 [==============================] - 19s 376us/step - loss: 0.0287 - acc: 0.8089


### Result

In [114]:
score = model.evaluate(x_test, y_test)
print('Loss for test data: %f'%(score[0]))
print('Accuracy: %f'%(score[1]))

10000/10000 [==============================] - 2s 230us/step
Loss for test data: 0.066390
Accuracy: 0.553700
